<a href="https://colab.research.google.com/github/Bumaza/Automatic-music-transcription/blob/master/pmt_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
PATH = '/content/drive/My Drive/AMT/'
DPATH = '/content/drive/My Drive/AMT/data352/'
!ls "/content/drive/My Drive/AMT/"
!pip install pretty_midi

In [0]:
MIN_MIDI_TONE = 21
MAX_MIDI_TONE = 108
NOTE_RANGE = MAX_MIDI_TONE - MIN_MIDI_TONE + 1
ALL_MIDI_TONES = 128

BIN_MULTIPLE = 4 # 3
BIN_PER_OCTAVE = 12 * BIN_MULTIPLE
N_BINS = NOTE_RANGE * BIN_MULTIPLE 

HYBRID_DIR = PATH + 'MAPS-orig/StbgTGd2/MUS/'
BOESENDORFER_IMPERIAL_DIR = PATH + 'MAPS-orig/AkPnBsdf/MUS/'
BECHSTEIN_DIR = PATH + 'MAPS-orig/AkPnBcht/MUS/'
CONCERT_GRAND_DIR = PATH + 'MAPS-orig/AkPnCGdD/MUS/'
STEINGRAEBER_DIR = PATH + 'MAPS-orig/AkPnStgb/MUS/'
STEINWAY_DIR =  PATH + 'MAPS-orig/SptkBGAm/MUS/'
STEINWAY_DIR2 =  PATH + 'MAPS-orig/SptkBGCl/MUS/'
REAL_PIANO_DIR = PATH + 'MAPS-orig/ENSTDkCl/MUS/'

# MAPS configuration
train_set = set(['AkPnBcht', 'AkPnCGdD', 'AkPnStgb', 'SptkBGAm'])
val_set = set(['SptkBGCl'])
test_set = set(['ENSTDkCl']) 

print(STEINGRAEBER_DIR)

MIDI_DIR = PATH + 'midi/' # CMPT datasets file
MP3_DIR = PATH + 'mp3/'  # CMPT datasets file

In [0]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn
import pretty_midi
from numpy import newaxis
import seaborn as sns
from matplotlib import colors 



MIN_LEVEL_DB = -100.0
REF_LEVEL_DB = 20.0
SAMPLE_RATE = 16000
HOP_LENGTH = 512
WIN_LENGTH = 1024
WINDOW_SIZE = 7

units = 125
layers = 3
dropout = 0.3
BATCH_SIZE = 256

# 1. DNN, 2. LSTM, 3. ConvNet, 4. ConvNet BNR
MODEL_TYPE = 5
FEATURES = 'cqt'


PLOT = True


def amp_to_db(x):
    return 20.0 * np.log10(np.maximum(1e-5, x))


def db_to_amp(x):
    return np.power(10.0, x * 0.05)


def normalize(S):
    return np.clip(S / -MIN_LEVEL_DB, -1.0, 0.0) + 1.0


def denormalize(S):
    return (np.clip(S, 0.0, 1.0) - 1.0) * -MIN_LEVEL_DB


def stft(y):
    """ Compute STFT for audio
     :param y:
     :return:
     """
    return librosa.stft(y=y, hop_length=HOP_LENGTH, win_length=WIN_LENGTH)


def istft(mag, phase):
    """ Compute inverse STFT
    :param mag:
    :param phase:
    :return:
    """
    stft_matrix = mag * np.exp(1j * phase)
    return librosa.istft(stft_matrix, hop_length=HOP_LENGTH, win_length=WIN_LENGTH)

def wav2spec(y):
    """ Transform wav file amplitude to
    time-frequency domain using stft algorithm
    :param y:
    :return:
    """
    D = stft(y)
    S = amp_to_db(np.abs(D)) - REF_LEVEL_DB
    S, D = normalize(S), np.angle(D)
    S, D = S.T, D.T  # to make [time, freq]
    return S, D


def spec2wav(spectrogram, phase):
    """
    :param spectrogram:
    :param phase:
    :return:
    """
    spectrogram, phase = spectrogram.T, phase.T
    # used during inference only
    # spectrogram: enhanced output
    # phase: use noisy input's phase, so no GLA is required
    S = db_to_amp(denormalize(spectrogram) + REF_LEVEL_DB)
    return istft(S, phase) 


def wav2cqt_spec(wav_file, windowed = False):
    """ Convert wav file to cqT spectogram
    :param wav_file:
    :return:
    """
    y, sr = librosa.load(os.path.join(MP3_DIR, wav_file), sr=SAMPLE_RATE)
    # Transpose so that the data is in [frame, bins] format.
    C = librosa.cqt(y, sr=SAMPLE_RATE, fmin=librosa.midi_to_hz(MIN_MIDI_TONE),
                    hop_length=HOP_LENGTH, bins_per_octave=BIN_PER_OCTAVE, n_bins=N_BINS).T
    C = np.absolute(C)

    if PLOT:
      librosa.display.specshow(librosa.amplitude_to_db(C.T, ref=np.max), y_axis='cqt_note', x_axis='time', sr=SAMPLE_RATE, hop_length=HOP_LENGTH, bins_per_octave=BIN_PER_OCTAVE)
      plt.title('constant-Q transform of ' + track)
      plt.colorbar(format='%+2.0f dB')
      plt.tight_layout()
      plt.show()


    #C = librosa.amplitude_to_db(C, ref=np.max)
    #C = sklearn.preprocessing.normalize(C)
    #minDB = np.min(C)

    if windowed:
        windows = [C[i:i + WINDOW_SIZE, :] for i in range(C.shape[0] - WINDOW_SIZE + 1)]
        return np.array(windows)

    return C

  
def wav2mel(wav_file):
  """Transforms the contents of a wav file into a series of mel spec frames."""
  y, _ = librosa.load(os.path.join(MP3_DIR, wav_file), sr=SAMPLE_RATE)

  mel = librosa.feature.melspectrogram(y, SAMPLE_RATE, hop_length=HOP_LENGTH, fmin=librosa.midi_to_hz(MIN_MIDI_TONE), n_mels=229)

  if PLOT:
      librosa.display.specshow(librosa.amplitude_to_db(mel, ref=np.max), y_axis='cqt_note', x_axis='time', sr=SAMPLE_RATE, hop_length=HOP_LENGTH, bins_per_octave=BIN_PER_OCTAVE)
      plt.title('Mel spectrum of ' + track)
      plt.colorbar(format='%+2.0f dB')
      plt.tight_layout()
      plt.show()

  # Transpose so that the data is in [frame, bins] format.
  mel = mel.T
  return mel 


def midi2labels(midi_file, times):
    """ Convert midi file to a piano roll
    :param midi_file:
    :param times:
    :return:
    """
    pm = pretty_midi.PrettyMIDI(os.path.join(MIDI_DIR, midi_file))
    piano_roll = pm.get_piano_roll(fs=100, times=times)[MIN_MIDI_TONE:MAX_MIDI_TONE + 1].T
    piano_roll[piano_roll > 0] = 1
    
    if PLOT:
      plot_predict(piano_roll)

    return piano_roll


def plot_predict(y, truth=False):
    cmap = colors.ListedColormap(['white', 'black'])

    fig = plt.figure(figsize=(20, 5))
    plt.title('Ground Truth' if truth else 'Predict')
    plt.imshow(y.T, aspect='auto', cmap=cmap) #, 
    plt.gca().invert_yaxis()
    fig.axes[0].set_xlabel('window')
    fig.axes[0].set_ylabel('note (MIDI code')
    plt.show()    
    
def postprocess(y_pred):

    # if not self.preprocessing:
    #     return y_pred

    y_pred = np.array(y_pred).round()
    y_pred[y_pred > 1] = 1


    changes = 0

    for note in range(y_pred.shape[1]):
        for frame in range(2, y_pred.shape[0] - 3):

            if list(y_pred[frame-1:frame+3, note]) == [1.0, 0.0, 0.0, 1.0]:
                y_pred[frame, note], y_pred[frame + 1, note] = 1, 1
                changes += 1

            if list(y_pred[frame-2:frame+4, note]) == [0.0, 0.0, 1.0, 1.0, 0.0, 0.0]:
                y_pred[frame, note], y_pred[frame + 1, note] = 0, 0
                changes += 1

            if list(y_pred[frame-1:frame+3, note]) == [0.0, 1.0, 0.0, 0.0]:
                y_pred[frame, note] = 0
                changes += 1

            if list(y_pred[frame-1:frame+3, note]) == [1.0, 0.0, 1.0, 1.0]:
                y_pred[frame, note] = 1
                changes += 1

    print('Total changes: {0}'.format(changes))
    return y_pred

In [0]:
#code for preprocessing CMPT dataset

from os import listdir

np.random.seed()

tracks = [x for x in listdir(MP3_DIR) if x.endswith('.wav')]
np.random.shuffle(tracks)

print(len(tracks), tracks[0])

i, length = 1, len(tracks)

X_all, y_all = None, None

pretty_midi.pretty_midi.MAX_TICK = 1e10

for track in tracks: 
    try:
        X = wav2cqt_spec(track)
        times = librosa.frames_to_time(np.arange(X.shape[0]), sr=SAMPLE_RATE, hop_length=HOP_LENGTH)
        y = midi2labels('{0}.mid'.format(track.split('.')[0]), times)

        if X_all is None or y_all is None:
            X_all, y_all = X, y
        elif X.shape[0] == y.shape[0]:
            X_all, y_all = np.concatenate((X_all, X)), np.concatenate((y_all, y))

        print('{0}/{1} {2} {3}.mid'.format(i, length, track, track.split('.')[0]), X.shape, '/', X_all.shape, y.shape, '/', y_all.shape)
        i += 1

    except FileNotFoundError as err:
        print(err)
    except Exception as err:
        print(err)

print(X_all.shape, y_all.shape)

np.save(PATH + MIDI_DIR.split('/')[-3] + '_input.npy', X_all)
np.save(PATH + MIDI_DIR.split('/')[-3] + '_output.npy', y_all)

In [0]:
#Loading CPMT dataset


X_train, y_train = np.load(DPATH+'train_input_cqt.npy')[:827025], np.load(PATH+'train_output_cqt.npy')[:827025]
X_validate, y_validate = np.load(DPATH+'val_input_cqt.npy')[:242604], np.load(PATH+'val_output_cqt.npy')[:242604]

min_train, max_train = X_train.min(axis=0), X_train.max(axis=0)
min_val, max_val = X_validate.min(axis=0), X_validate.max(axis=0)

X_train = (X_train - min_train) / (max_train - min_train)
X_validate = (X_validate - min_val) / (max_val - min_val)

print(X_train.shape, y_train.shape)
print(X_validate.shape, y_validate.shape) 

In [0]:
#Loading MIMAS piano part 
X_train, y_train = np.load(DPATH+'folk_piano_train_input.npy'), np.load(DPATH+'folk_piano_train_output.npy')
X_validate, y_validate = np.load(DPATH+'folk_piano_val_input.npy'), np.load(DPATH+'folk_piano_val_output.npy')

min_train, max_train = X_train.min(axis=0), X_train.max(axis=0)
min_val, max_val = X_validate.min(axis=0), X_validate.max(axis=0)

X_train = (X_train - min_train) / (max_train - min_train)
X_validate = (X_validate - min_val) / (max_val - min_val)

print(X_train.shape, y_train.shape)
print(X_validate.shape, y_validate.shape)

In [0]:
# Loading MAPS dataset TRAIN 
X_train, y_trian = None, None

for data in train_set:
  if X_train is None:
    X_train, y_train = np.load(DPATH+data+'_input.npy'), np.load(DPATH+data+'_output.npy')
  else:
    X_train, y_train = np.concatenate((X_train, np.load(DPATH+data+'_input.npy'))), np.concatenate((y_train, np.load(DPATH+data+'_output.npy')))

min_train, max_train = X_train.min(axis=0), X_train.max(axis=0)
X_train = (X_train - min_train) / (max_train - min_train)

print('TRAIN DATA: ', X_train.shape, y_train.shape)

X_validate, y_validate = None, None

for data in val_set:
  if X_validate is None:
    X_validate, y_validate = np.load(DPATH+data+'_input.npy'), np.load(DPATH+data+'_output.npy')
  else:
    X_validate, y_validate = np.concatenate((X_validate, np.load(DPATH+data+'_input.npy'))), np.concatenate((y_validate, np.load(DPATH+data+'_output.npy')))

min_val, max_val = X_validate.min(axis=0), X_validate.max(axis=0)
X_validate = (X_validate - min_val) / (max_val - min_val)

print('VALIDATION DATA: ', X_validate.shape, y_validate.shape)

#MAPS dataset TEST
X_test, y_test = None, None

for data in test_set:
  if X_test is None:
    X_test, y_test = np.load(DPATH+data+'_input.npy'), np.load(DPATH+data+'_output.npy')
  else:
    X_test, y_test = np.concatenate((X_test, np.load(DPATH+data+'_input.npy'))), np.concatenate((y_test, np.load(DPATH+data+'_output.npy')))

min_test, max_test = X_test.min(axis=0), X_test.max(axis=0)
X_test = (X_test - min_test) / (max_test - min_test)

print('TEST DATA: ', X_test.shape, y_test.shape)

In [0]:
# utility callback for ploting posteigroms during training process 

import sklearn
from keras import backend as K
from keras.callbacks import Callback
from sklearn.metrics import precision_recall_fscore_support

class CheckMidi(Callback):

  def __init__(self, data):
    self.data = data
    self.X = None

    PRED_SIZE = 10000

    for data in test_set:
      self.X = np.load(PATH+data+'_input.npy')

    min_test, max_test = self.X.min(axis=0), self.X.max(axis=0)
    self.X = (self.X - min_test) / (max_test - min_test)

    if MODEL_TYPE > 2:
      self.X = np.array([self.X[i:i+WINDOW_SIZE, :] for i in range(0, PRED_SIZE - WINDOW_SIZE + 1)])[..., newaxis]
    elif MODEL_TYPE == 2: #INPUT FOR LSTM
      self.X = np.array([self.X[i:i+BATCH_SIZE, :] for i in range(0, PRED_SIZE - BATCH_SIZE + 1, BATCH_SIZE)])

    print('Check MIDI: ',  self.X.shape)

  def on_epoch_end(self, epoch, logs={}):
    y_pred = self.model.predict(self.X, batch_size=BATCH_SIZE, verbose=1)
    if MODEL_TYPE == 2: #LSTM
      y = None
      for i in range(0, len(y_pred)):
          if y is None:
            y = y_pred[i]
          else:
            y = np.concatenate((y, y_pred[i]))
      print(y.shape)
      y_pred = y

    plot_predict(y_pred)

In [0]:
# Data generator for efficiently manipulation with RAM memory

class DataGenerator:

  def __init__(self, data):
    self.step = 0
    self.data = data
    self.current_file = 0
    self.audio, self.midi = None, None

    self.count_steps()
    self.load_normalize_slice()


  def count_steps(self):
    for d in self.data:
      m = np.load(PATH+d+'_output.npy')
      s = len(m) + 1 - WINDOW_SIZE
      self.step += s // BATCH_SIZE


  def load_normalize_slice(self):
    self.audio, self.midi = np.load(PATH+self.data[self.current_file]+'_input.npy'), np.load(PATH+self.data[self.current_file]+'_output.npy')
    ''' normalize data '''
    min_train, max_train = self.audio.min(axis=0), self.audio.max(axis=0)
    self.audio = (self.audio - min_train) / (max_train - min_train)

    ''' slice to windows '''
    SIZE = len(self.audio)
    self.audio = np.array([self.audio[j:j+WINDOW_SIZE, :] for j in range(0, SIZE - WINDOW_SIZE + 1)])[..., newaxis]
    self.midi = self.midi[:SIZE + 1 - WINDOW_SIZE,:]

    print('DataGenerator: ', self.audio.shape, self.midi.shape)


  def steps(self):
    return self.step

  def next(self):
    i = 0
    while True:
      
      if (i+1) * BATCH_SIZE > self.audio.shape[0]:
        
        X, y = self.audio[i * BATCH_SIZE:], self.midi[i * BATCH_SIZE:]
        i = 0

        if len(self.data) > 1:
          self.current_file = (self.current_file + 1) % len(self.data)
          self.load_normalize_slice()

      else:
        X, y = self.audio[i * BATCH_SIZE:(i+1)* BATCH_SIZE], self.midi[i * BATCH_SIZE: (i+1) * BATCH_SIZE]
        i += 1

      yield X, y



In [0]:
import sklearn
from keras import backend as K
from keras.callbacks import Callback

class LinearDecay(Callback):
   
    def __init__(self, initial_lr, epochs):
        super(LinearDecay, self).__init__()
        self.initial_lr = initial_lr
        self.decay = initial_lr/epochs

    def on_epoch_begin(self, epoch, logs={}):
        new_lr = self.initial_lr - self.decay*epoch
        print("ld: learning rate is now "+str(new_lr))
        K.set_value(self.model.optimizer.lr, new_lr)

In [0]:
#DNN & LSTM models implementation
from keras.optimizers import Adam, Adadelta, SGD
from keras.models import Sequential, Model
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Dense, Flatten, Reshape, Input, LSTM, add
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
from keras.utils.vis_utils import plot_model

MODEL_PATH = PATH + 'Sigtia/models/'

dataset = 'maps' # 'cmpt', 'mimas'
optimizer = 'adam' #sgd

model = Sequential()
model_file = MODEL_PATH + 'dnn_{0}_{1}.hdf5'.format(layers, optimizer)

MODEL_TYPE = 5
units = 125
layers = 3



if MODEL_TYPE == 1: #229
  model.add(Dense(units, input_shape=(N_BINS, ), kernel_initializer='normal', activation='relu'))
  model.add(Dropout(dropout))

  for _ in range(layers - 1):
    model.add(Dense(units, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout))

  model.add(Dense(NOTE_RANGE, kernel_initializer='normal', activation='sigmoid'))

elif MODEL_TYPE == 2: # RNN
  # input 3d array(batch_size, time_steps, seq_len)
  model_file = MODEL_PATH + 'lstm_{0}_{1}.hdf5'.format(layers, optimizer)
  model.add(LSTM(units, input_shape=(BATCH_SIZE, N_BINS), return_sequences=True, kernel_initializer='normal', activation='tanh'))
  model.add(Dropout(dropout))

  for _ in range(1):
    model.add(LSTM(units, return_sequences=True, kernel_initializer='normal', activation='tanh'))
    model.add(Dropout(dropout))

  # return_sequence -> TRUE  - output 3d array (batch_size, time_steps, units)
  # return_sequence -> FALSE - output 2d array (batch_size, units)
  model.add(Dense(NOTE_RANGE, kernel_initializer='normal', activation='relu'))

elif MODEL_TYPE == 3:
  model_file = MODEL_PATH + 'convnet_{0}_{1}.hdf5'.format(dataset, optimizer)
  # 1. CNN layer
  model.add(Conv2D(50, (5, 25), input_shape=(WINDOW_SIZE, N_BINS, 1), activation='tanh'))
  model.add(Dropout(0.5))
  model.add(MaxPooling2D(pool_size=(1, 3)))

  # 2. CNN layer
  model.add(Conv2D(50, (3, 5), activation='tanh'))
  model.add(Dropout(0.5))
  model.add(MaxPooling2D(pool_size=(1, 3)))

  # Flatten layer
  model.add(Flatten())
  model.add(Dense(1000, activation='sigmoid'))
  model.add(Dropout(0.5))

  # Final layer
  model.add(Dense(200, activation='sigmoid'))
  model.add(Dropout(0.5))
  model.add(Dense(NOTE_RANGE, activation='sigmoid'))

elif MODEL_TYPE == 4:
  model_file = MODEL_PATH + 'convnetBNR_{0}_{1}.hdf5'.format(dataset)
  # 1. CNN layer
  model.add(Conv2D(50, (5, 25), input_shape=(WINDOW_SIZE, N_BINS, 1)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.25))
  model.add(MaxPooling2D(pool_size=(1, 2)))

  # 2. CNN layer
  model.add(Conv2D(50, (3, 5)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.25))
  model.add(MaxPooling2D(pool_size=(1, 2)))

  # Flatten layer
  model.add(Flatten())
  model.add(Dense(1000))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.25))

  # Final layer
  model.add(Dense(200))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(NOTE_RANGE, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer=optimizer)


decay = LinearDecay(0.001, 1000)
checkpoint = ModelCheckpoint(filepath=model_file, verbose=1, save_best_only=False)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')
checkMidi = CheckMidi(list(test_set))

callbacks = [checkpoint, earlystop, decay, checkMidi]

model.summary()

#plot_model(model, to_file=PATH + 'model_{0}.png'.format(MODEL_TYPE), show_shapes=True)

In [0]:
# windows for ConvNets
from numpy import newaxis

SIZE = len(X_train)

X_train = np.array([X_train[i:i+WINDOW_SIZE, :] for i in range(0, SIZE - WINDOW_SIZE + 1)])[..., newaxis]
y_train = y_train[:SIZE + 1 - WINDOW_SIZE,:]

print(X_train.shape)
print(y_train.shape)

SIZE = len(X_validate)

X_validate = np.array([X_validate[i:i+WINDOW_SIZE, :] for i in range(0, SIZE - WINDOW_SIZE + 1)])[..., newaxis]
y_validate = y_validate[:SIZE + 1 - WINDOW_SIZE,:]

print(X_validate.shape)
print(y_validate.shape)

SIZE = len(X_test)

X_test = np.array([X_test[i:i+WINDOW_SIZE, :] for i in range(0, SIZE - WINDOW_SIZE + 1)])[..., newaxis]
y_test = y_test[:SIZE + 1 - WINDOW_SIZE,:]

print(X_validate.shape)
print(y_validate.shape)



In [0]:
# LSTM INPUT

X_train = np.array([X_train[i:i+BATCH_SIZE, :] for i in range(0, len(X_train) - BATCH_SIZE + 1, BATCH_SIZE)])
X_validate = np.array([X_validate[i:i+BATCH_SIZE, :] for i in range(0, len(X_validate) - BATCH_SIZE + 1, BATCH_SIZE)])

y_train = np.array([y_train[i:i+BATCH_SIZE, :] for i in range(0, len(y_train) - BATCH_SIZE + 1, BATCH_SIZE)])
y_validate = np.array([y_validate[i:i+BATCH_SIZE, :] for i in range(0, len(y_validate) - BATCH_SIZE + 1, BATCH_SIZE)])

print(X_train.shape, y_train.shape)
print(X_validate.shape, y_validate.shape)

In [0]:
#FIT generator

trainGenerator = DataGenerator(list(train_set))
valGenerator = DataGenerator(list(val_set))

save = model.fit_generator(trainGenerator.next(), trainGenerator.steps(), epochs=1000, verbose=1, validation_data=valGenerator.next(), validation_steps=valGenerator.steps(), callbacks=callbacks)


In [0]:
BATCH_SIZE = 100
EPOCHS = 1000

save = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=10, validation_data=(X_validate, y_validate) ,verbose=1, callbacks=callbacks)

import json


with open(HISTORY_PATH +'history{0}_{1}_{2}.json'.format(MODEL_TYPE, dataset, optimizer), 'w') as f:
    json.dump(save.history, f)

In [0]:
def metrics(y_pred, y_test):
  a = min(len(y_pred), len(y_test))
  y_test = y_test[:a]
  y_pred = y_pred[:a]

  #y_pred = np.array()

  TP = np.count_nonzero(np.logical_and( y_pred == 1, y_test == 1 ))
  FN = np.count_nonzero(np.logical_and( y_pred == 0, y_test == 1 ))
  FP = np.count_nonzero(np.logical_and( y_pred == 1, y_test == 0 ))

  A, F, R, P = 0, 0, 0, 0

  if (TP + FN) > 0:
    R, P = TP / float(TP + FN), TP / float(TP + FP)
    A = 100 * TP / float(TP + FP + FN)
    F = 100 * 2 * P * R / (P+R)
    

  print('Precision: ', P)
  print('Recall: ', R)
  print('F-measure: ', F)
  print('Accuracy: ', A)

In [0]:
#  Evaluation
PRED_SIZE = len(X_test)

MODEL_TYPE = 1

print('TEST DATA: ', X_test.shape, y_test.shape)

from keras.models import Model, load_model

# Sigtia/models/

model = load_model(PATH +'My 352/models/cnn2_folk_adam.hdf5') #lstm_maps_3_cqt

y_pred = model.predict(X_test[:PRED_SIZE], batch_size=256, verbose=1)

if MODEL_TYPE == 2: #LSTM
  y = None
  for i in range(0, len(y_pred)):
      if y is None:
        y = y_pred[i]
      else:
        y = np.concatenate((y, y_pred[i]))
  print(y.shape)
  y_pred = y


plot_predict(y_pred)
y_pred = np.array(y_pred).round()

metrics(y_pred, y_test)

y_pred = postprocess(y_pred)

metrics(y_pred, y_test)
plot_predict(y_pred)

plot_predict(y_test[:PRED_SIZE])